# Installing dependencies and loading data

In [93]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests
import json

In [30]:
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

In [31]:
merged_data = transactions.merge(customers, on="CustomerID").merge(products, on="ProductID")

# Data Transformations

In [32]:
# Convert date columns to datetime
merged_data['TransactionDate'] = pd.to_datetime(merged_data['TransactionDate'])
merged_data['Month'] = merged_data['TransactionDate'].dt.month_name()

In [33]:
# Check for missing values
print("\nMissing Values:")
print("Customers:", customers.isnull().sum())
print("Products:", products.isnull().sum())
print("Transactions:", transactions.isnull().sum())


Missing Values:
Customers: CustomerID      0
CustomerName    0
Region          0
SignupDate      0
dtype: int64
Products: ProductID      0
ProductName    0
Category       0
Price          0
dtype: int64
Transactions: TransactionID      0
CustomerID         0
ProductID          0
TransactionDate    0
Quantity           0
TotalValue         0
Price              0
dtype: int64


In [159]:
merged_data.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,CustomerName,Region,SignupDate,ProductName,Category,Price,Month
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68,August
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68,May
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68,April
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68,March
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68,March


In [166]:
# Extract month from timestamp

merged_data['TransactionDate'] = pd.to_datetime(merged_data['TransactionDate'], errors='coerce')
# print(merged_data['TransactionDate'].isnull().sum())
merged_data['Month'] = merged_data['TransactionDate'].dt.month_name()
merged_data['Month'].unique()  

array(['August', 'May', 'April', 'March', 'December', 'November',
       'September', 'January', 'October', 'June', 'July', 'February'],
      dtype=object)

In [218]:
# Region Sales

region_sales = merged_data.groupby('Region')['TotalValue'].sum().reset_index()
region_sales

,Region,TotalValue
0,Asia,152074.97
1,Europe,166254.63
2,North America,152313.40
3,South America,219352.56


In [169]:
# Group data by Category and sum TotalValue

category_sales = merged_data.groupby('Category')['TotalValue'].sum().reset_index()
category_sales = category_sales.sort_values('TotalValue', ascending=False)
category_sales

,Category,TotalValue
0,Books,192147.47
2,Electronics,180783.50
1,Clothing,166170.66
3,Home Decor,150893.93


In [168]:
# Monthly Sales

monthly_sales = merged_data.groupby('Month')['TotalValue'].sum().reset_index()
monthly_sales['Month'] = pd.Categorical(
    monthly_sales['Month'],
    categories=[
        'January', 'February', 'March', 'April', 'May', 'June',
        'July', 'August', 'September', 'October', 'November', 'December'
    ],
    ordered=True
)
monthly_sales = monthly_sales.sort_values('Month')
monthly_sales

,Month,TotalValue
4,January,66376.39
3,February,51459.27
7,March,47828.73
0,April,57519.06
8,May,64527.74
6,June,48771.18
5,July,71366.39
1,August,63436.74
11,September,70603.75
10,October,47063.22


In [220]:
# Top 10 Products

product_sales = merged_data.groupby('ProductName')['TotalValue'].sum().reset_index()
top_10_products = product_sales.nlargest(10, 'TotalValue')
top_10_products

,ProductName,TotalValue
9,ActiveWear Smartwatch,39096.97
47,SoundWave Headphones,25211.64
52,SoundWave Novel,24507.90
4,ActiveWear Jacket,22712.56
6,ActiveWear Rug,22314.43
58,TechPro Headphones,19513.80
15,BookWorld Cookbook,19221.99
21,BookWorld Sweater,18743.79
64,TechPro Textbook,18267.96
2,ActiveWear Cookware Set,18083.73


# Exploratory Data Analysis

### Total Sales by Region

In [219]:
fig = px.area(region_sales, 
              x='Region', 
              y='TotalValue', 
              title="Total Sales by Region", 
              color='Region', 
              color_discrete_sequence=px.colors.sequential.Tealgrn,  
              labels={'Total Sales by Region': 'Total Sales by Region', 'Region': 'Continent'},
              hover_data={'TotalValue': True, 'Region': True})

fig.show()

### Category-wise Sales Distribution

In [162]:
fig_category_sales = px.bar(
    category_sales,
    y='Category',  
    x='TotalValue',  
    title='Category-Wise Sales Distribution',
    labels={'TotalValue': 'Sales Value', 'Category': 'Product Category'},
    color='Category',  
    color_discrete_sequence=px.colors.sequential.Agsunset,  
    text='TotalValue',  
    barmode='stack'     
)

fig_category_sales.update_layout(
    title_font_size=24,
    title_x=0.5,  
    xaxis_title='Total Sales Value',
    yaxis_title='Product Category',
    showlegend=True,  
    plot_bgcolor='rgba(0,0,0,0)', 
)

fig_category_sales.update_traces(textposition='inside', texttemplate='%{text:.2f}')

fig_category_sales.show()


### Monthly Sales Trend

In [170]:
fig_monthly_sales = px.line(
    monthly_sales,
    x='Month',
    y='TotalValue',
     markers=True,
    title='Monthly Sales Trend'
)
fig_monthly_sales.show()

### Price vs Quantity Sold by Category

In [197]:
fig_price_quantity = px.scatter(
    merged_data,
    x='Price',  
    y='Quantity',
    color='Category',
    size='TotalValue',
    hover_name='ProductName',
    title='Price vs Quantity Sold by Category',
    color_discrete_sequence=px.colors.sequential.Burg

)

fig_price_quantity.show()


### Price Distribution by Category

In [136]:
fig_box_price = px.box(
    merged_data,
    x='Category',
    y='Price',
    color='Category',
    title="Price Distribution by Category",
    labels={'Price': 'Product Price', 'Category': 'Product Category'},
    color_discrete_sequence=px.colors.sequential.Viridis  
)
fig_box_price.show()


### Top 10 Products by Total Sales

In [213]:
fig_top_products = px.bar(
    top_10_products,
    x='ProductName',
    y='TotalValue',
    color='ProductName',
    title='Top 10 Products by Total Sales',
    color_discrete_sequence=px.colors.sequential.Burg[::-1],  
    text_auto=True
)

fig_top_products.update_layout(
    xaxis_title="Product Name", 
    yaxis_title="Total Sales (USD)"
)




# Insights

#### 1. Total Sales by Region : 
- Customers from South America show the highest transaction count, leading the total sales by 6.79%, followed by Europe, Asia and North America, thereby presenting the greatest market potential based on current sales.

#### 2.  Category-wise Sales Distribution : 
- The Books category has the highest sales value, exceeding 192,000, followed by Electronics closely around 180,000.
- This might imply that expanding the Electronics and Clothing product lines might drive additional sales growth.

#### 3. Monthly Sales Trend
- There is a fluctuation in the month-on-month sales demonstrating a non-linear trend. 
- The peak sales happen in July and September. 
- The sharp dip in sales in June and October could be an indicator of seasonal factors.
- Sales are high in December and January, which can be attributed to the holiday season.

#### 4. Price vs Quantity Sold by Category
- Quantity sold generally decreases as price increases across all categories. 
- Electronics consistently have higher quantity sold compared to other categories, even at higher price points.
- Home Decor items seem to be priced lower than Electronics and Books, with a concentration of sales in the lower price range.

#### 5. Price Distribution by Category
- Electronics show a higher median price as compared to others, with a compact interquartile range, suggesting price variability.
- Home Decor and Books exhibit similar median prices, but Books exhibit greater price spread.
- Clothing has the lowest median price and wider range, indicating variability.

#### 6. Top 10 Products by Total Sales
- ActiveWear Smartwatch is the top-selling product. Its total value represents approximately 19.1% of the combined total value of the top 10 products.
- ActiveWear Jacket, Rug, and Cookware Set also rank highly, showcasing the popularity of the ActiveWear product line. Collectively, these four ActiveWear products contribute approximately 37.1% to the combined total value.
- There is a considerable difference between the top-selling product (ActiveWear Smartwatch) and the lowest (ActiveWear Cookware Set).The Smartwatch's total value is more than 2 times that of the Cookware Set.